# TranferEval class for evaluating transferability between models

Example of usage of the TransferBench library to evaluate the transferability of adversarial attacks on different models and datasets.

In [ ]:
%cd ..

In [ ]:
# case 1: Evaluate transferability using default scenarios, on torchvision models

from transferbench.evaluate_transferability import TransferEval

evaluator = TransferEval("resnet50",
[
    "resnet18",
    "resnet101",
    "resnet152"
])
result = evaluator.run(batch_size=4, device="cuda:1")
print(result)

In [ ]:
# case 2: Evaluate transferability using custom scenarios, on torchvision models

from transferbench.attacks import BaseHyperParameters
from transferbench.evaluate_transferability import TransferEval
from transferbench.utils.scenarios import TransferScenario

myscenario = TransferScenario(
    hp=BaseHyperParameters(eps=0.3, p=2, maximum_queries=10),
    attack_step="NaiveAvg",
    dataset="ImageNetT",
)

evaluator = TransferEval(
    "resnet50",
[
    "resnet18",
    "resnet101",
    "resnet152"
],
)
evaluator.set_scenarios(myscenario, "oneshot")
result = evaluator.run(batch_size=4, device="cuda:1")


In [ ]:
import torch
from torchvision import datasets, transforms
from transferbench.attacks import BaseHyperParameters
from transferbench.evaluate_transferability import TransferEval
from transferbench.models.utils import add_normalization
from transferbench.utils.scenarios import TransferScenario

# Load a dataset
transform = transforms.Compose([transforms.ToTensor()])
cifar100 = datasets.CIFAR100(
    root="./data/datasets",
    train=False,
    download=True,
    transform=transform,
)
cifar100_mean = [0.5, 0.5, 0.5]
cifar100_std = [1.0, 1.0, 1.0]

REPO_LINK = "chenyaofo/pytorch-cifar-models"
# Load models and normalize them
get_model = lambda model: torch.hub.load(
    REPO_LINK, "cifar100_" + model, pretrained=True
)

# Use the dataset in the TransferEval
victim_model = add_normalization(get_model("resnet56"), cifar100_mean, cifar100_std)
surrgoate_models = [
    add_normalization(get_model("vgg11_bn"), cifar100_mean, cifar100_std),
    add_normalization(get_model("vgg13_bn"), cifar100_mean, cifar100_std),
    add_normalization(get_model("vgg16_bn"), cifar100_mean, cifar100_std),
    add_normalization(get_model("vgg19_bn"), cifar100_mean, cifar100_std),
]

evaluator = TransferEval(
    victim_model,
    surrgoate_models,
)
myscenario = TransferScenario(
    hp=BaseHyperParameters(eps=0.3, p=2, maximum_queries=10),
    attack_step="NaiveAvg",
    dataset=cifar100,
)
evaluator.set_scenarios(myscenario)
result = evaluator.run(batch_size=4, device="cuda:1")
print(result)
